## Parsing Mortality Data

### Setup and read in file

Documentation for 2018 datafile: https://www.cdc.gov/nchs/data/dvs/Multiple_Cause_Record_Layout_2018-508.pdf

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 75)

In [ ]:
# 2018 raw data
file = "raw_data/Mort2018US.PubUse.txt"

In [ ]:
lines = open(file, "r").readlines()

In [ ]:
lines = [line.rstrip("\n") for line in lines]

In [ ]:
year_2018 = pd.DataFrame(lines)

In [ ]:
year_2018.columns = ["line_str"]
year_2018.index.name = "line_num"

In [ ]:
year_2018.head()

,line_str
line_num,
0,3 3101 F1046 351507 1D2 2018U7CN A419023 010 37 0311J189 21J869 31A419 03 A419 J189 J869 01 11 100 601
1,1 3101 M1028 311105 7S4 2018N1BN 9 V475392 114 38 0511S099 12V475 13T099 14T149 15S021 05 V475 S021 S099 T099 T149 03 23 100 803
2,1 6101 M1093 442411 6D3 2018U7BN C61 113 033 11 0111C61 01 C61 01 11 100 601
3,1 2101 M1090 442411 1W7 2018U7CN A419023 010 37 0311R688 21A419 61I509 03 A419 I509 R688 01 11 100 601
4,2 4101 F1031 321205 1S2 2018U7CN K701298 094 30 0911G934 21K701 22R579 31K769 32R688 41D689 61K703 62D649 63E880 08 K701 D649 D689 E880 G934 K703 R579 R688 03 23 100 803


In [ ]:
year_2018.count()

line_str    2846305
dtype: int64

In [ ]:
year_2018[year_2018['line_str'].str.len()==490].count() #all rows are 490 characters long

line_str    2846305
dtype: int64

### Copy slices of line_str into new cols

In [ ]:
year_2018["resident_status"] = year_2018["line_str"].map(lambda x: x[19])
year_2018["education"] = year_2018["line_str"].map(lambda x: x[60:63]).str.strip()
year_2018["education_flag"] = year_2018["line_str"].map(lambda x: x[63])
year_2018["death_month"] = year_2018["line_str"].map(lambda x: x[64:66])
year_2018["sex"] = year_2018["line_str"].map(lambda x: x[68])

In [ ]:
year_2018["age_flag"] = year_2018["line_str"].map(lambda x: x[69])
year_2018["age"] = year_2018["line_str"].map(lambda x: x[70:73]).str.strip()
year_2018["age_sub_flag"] = year_2018["line_str"].map(lambda x: x[73]).replace(r'^\s*$', np.nan, regex=True)
year_2018["age_recode_52"] = year_2018["line_str"].map(lambda x: x[74:76]).str.strip()
year_2018["age_recode_27"] = year_2018["line_str"].map(lambda x: x[76:78]).str.strip()
year_2018["age_recode_12"] = year_2018["line_str"].map(lambda x: x[78:80]).str.strip()
year_2018["infant_age_recode_22"] = year_2018["line_str"].map(lambda x: x[80:82]).str.strip().replace(r'^\s*$', np.nan, regex=True)

In [ ]:
year_2018["place_of_death"] = year_2018["line_str"].map(lambda x: x[82])
year_2018["marital_status"] = year_2018["line_str"].map(lambda x: x[83])
year_2018["death_day_of_week"] = year_2018["line_str"].map(lambda x: x[84])
year_2018["death_year"] = year_2018["line_str"].map(lambda x: x[101:105])

In [ ]:
year_2018["work_injury"] = year_2018["line_str"].map(lambda x: x[105])
year_2018["death_manner"] = year_2018["line_str"].map(lambda x: x[106]).replace(r'^\s*$', np.nan, regex=True)
year_2018["disposition_method"] = year_2018["line_str"].map(lambda x: x[107])
year_2018["autopsy"] = year_2018["line_str"].map(lambda x: x[108])

In [ ]:
year_2018["activity_code"] = year_2018["line_str"].map(lambda x: x[143]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["place_of_injury"] = year_2018["line_str"].map(lambda x: x[144]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["ucd"] = year_2018["line_str"].map(lambda x: x[145:149]).str.strip()
year_2018["ucd_recode_358"] = year_2018["line_str"].map(lambda x: x[149:152]).str.strip()
year_2018["ucd_recode_113"] = year_2018["line_str"].map(lambda x: x[153:156]).str.strip()

In [ ]:
year_2018["ucd_infant_recode_130"] = year_2018["line_str"].map(lambda x: x[156:159]).replace(r'^\s*$', np.nan, regex=True)
year_2018["ucd_recode_39"] = year_2018["line_str"].map(lambda x: x[159:161])

In [ ]:
# E represents "Entity Axis" (written by the person who completed death certificate)
year_2018["mcd_count_E"] = year_2018["line_str"].map(lambda x: x[162:164]).str.strip()
year_2018["mcd_1_E"] = year_2018["line_str"].map(lambda x: x[166:170]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["mcd_2_E"] = year_2018["line_str"].map(lambda x: x[173:177]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["mcd_3_E"] = year_2018["line_str"].map(lambda x: x[180:184]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["mcd_4_E"] = year_2018["line_str"].map(lambda x: x[187:191]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["mcd_5_E"] = year_2018["line_str"].map(lambda x: x[194:198]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["mcd_6_E"] = year_2018["line_str"].map(lambda x: x[201:205]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["mcd_7_E"] = year_2018["line_str"].map(lambda x: x[208:212]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["mcd_8_E"] = year_2018["line_str"].map(lambda x: x[215:219]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["mcd_9_E"] = year_2018["line_str"].map(lambda x: x[222:226]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["mcd_10_E"] = year_2018["line_str"].map(lambda x: x[229:233]).str.strip().replace(r'^\s*$', np.nan, regex=True)

In [ ]:
year_2018["mcd_11_E"] = year_2018["line_str"].map(lambda x: x[236:240]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["mcd_12_E"] = year_2018["line_str"].map(lambda x: x[243:247]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["mcd_13_E"] = year_2018["line_str"].map(lambda x: x[250:254]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["mcd_14_E"] = year_2018["line_str"].map(lambda x: x[257:261]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["mcd_15_E"] = year_2018["line_str"].map(lambda x: x[264:268]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["mcd_16_E"] = year_2018["line_str"].map(lambda x: x[271:275]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["mcd_17_E"] = year_2018["line_str"].map(lambda x: x[278:282]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["mcd_18_E"] = year_2018["line_str"].map(lambda x: x[285:289]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["mcd_19_E"] = year_2018["line_str"].map(lambda x: x[292:296]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["mcd_20_E"] = year_2018["line_str"].map(lambda x: x[299:303]).str.strip().replace(r'^\s*$', np.nan, regex=True)

In [ ]:
# R represents "Record Axis" (cleaned to remove inconsistencies)
year_2018["mcd_count_R"] = year_2018["line_str"].map(lambda x: x[340:342]).str.strip()
year_2018["mcd_1_R"] = year_2018["line_str"].map(lambda x: x[343:347]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["mcd_2_R"] = year_2018["line_str"].map(lambda x: x[348:352]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["mcd_3_R"] = year_2018["line_str"].map(lambda x: x[353:357]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["mcd_4_R"] = year_2018["line_str"].map(lambda x: x[358:362]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["mcd_5_R"] = year_2018["line_str"].map(lambda x: x[363:367]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["mcd_6_R"] = year_2018["line_str"].map(lambda x: x[368:372]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["mcd_7_R"] = year_2018["line_str"].map(lambda x: x[373:377]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["mcd_8_R"] = year_2018["line_str"].map(lambda x: x[378:382]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["mcd_9_R"] = year_2018["line_str"].map(lambda x: x[383:387]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["mcd_10_R"] = year_2018["line_str"].map(lambda x: x[388:392]).str.strip().replace(r'^\s*$', np.nan, regex=True)

In [ ]:
year_2018["mcd_11_R"] = year_2018["line_str"].map(lambda x: x[393:397]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["mcd_12_R"] = year_2018["line_str"].map(lambda x: x[398:402]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["mcd_13_R"] = year_2018["line_str"].map(lambda x: x[403:407]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["mcd_14_R"] = year_2018["line_str"].map(lambda x: x[408:412]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["mcd_15_R"] = year_2018["line_str"].map(lambda x: x[413:417]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["mcd_16_R"] = year_2018["line_str"].map(lambda x: x[418:422]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["mcd_17_R"] = year_2018["line_str"].map(lambda x: x[423:427]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["mcd_18_R"] = year_2018["line_str"].map(lambda x: x[428:432]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["mcd_19_R"] = year_2018["line_str"].map(lambda x: x[433:437]).str.strip().replace(r'^\s*$', np.nan, regex=True)
year_2018["mcd_20_R"] = year_2018["line_str"].map(lambda x: x[438:442]).str.strip().replace(r'^\s*$', np.nan, regex=True)

In [ ]:
year_2018["race"] = year_2018["line_str"].map(lambda x: x[444:446]).str.strip()
year_2018["race_bridged_flag"] = year_2018["line_str"].map(lambda x: x[446]).replace(r'^\s*$', np.nan, regex=True)
year_2018["race_imputed_flag"] = year_2018["line_str"].map(lambda x: x[447]).replace(r'^\s*$', np.nan, regex=True)
year_2018["race_recode_3"] = year_2018["line_str"].map(lambda x: x[448])
year_2018["race_recode_5"] = year_2018["line_str"].map(lambda x: x[449])

In [ ]:
year_2018["hispanic"] = year_2018["line_str"].map(lambda x: x[483:486]).str.strip()
year_2018["hispanic_race_recode"] = year_2018["line_str"].map(lambda x: x[487])
year_2018["race_recode_40"] = year_2018["line_str"].map(lambda x: x[488:490]).str.strip()

### Visualize full df

In [ ]:
year_2018.head()

,line_str,resident_status,education,education_flag,death_month,sex,age_flag,age,age_sub_flag,age_recode_52,age_recode_27,age_recode_12,infant_age_recode_22,place_of_death,marital_status,death_day_of_week,death_year,work_injury,death_manner,disposition_method,autopsy,activity_code,place_of_injury,ucd,ucd_recode_358,ucd_recode_113,ucd_infant_recode_130,ucd_recode_39,mcd_count_E,mcd_1_E,mcd_2_E,mcd_3_E,mcd_4_E,mcd_5_E,mcd_6_E,mcd_7_E,mcd_8_E,mcd_9_E,mcd_10_E,mcd_11_E,mcd_12_E,mcd_13_E,mcd_14_E,mcd_15_E,mcd_16_E,mcd_17_E,mcd_18_E,mcd_19_E,mcd_20_E,mcd_count_R,mcd_1_R,mcd_2_R,mcd_3_R,mcd_4_R,mcd_5_R,mcd_6_R,mcd_7_R,mcd_8_R,mcd_9_R,mcd_10_R,mcd_11_R,mcd_12_R,mcd_13_R,mcd_14_R,mcd_15_R,mcd_16_R,mcd_17_R,mcd_18_R,mcd_19_R,mcd_20_R,race,race_bridged_flag,race_imputed_flag,race_recode_3,race_recode_5,hispanic,hispanic_race_recode,race_recode_40
line_num,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,3 3101 F1046 351507 1D2 2018U7CN A419023 010 37 0311J189 21J869 31A419 03 A419 J189 J869 01 11 100 601,3,3,1,01,F,1,046,NaN,35,15,07,NaN,1,D,2,2018,U,7,C,N,NaN,NaN,A419,023,010,NaN,37,03,J189,J869,A419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03,A419,J189,J869,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01,NaN,NaN,1,1,100,6,01
1,1 3101 M1028 311105 7S4 2018N1BN 9 V475392 114 38 0511S099 12V475 13T099 14T149 15S021 05 V475 S021 S099 T099 T149 03 23 100 803,1,3,1,01,M,1,028,NaN,31,11,05,NaN,7,S,4,2018,N,1,B,N,9,NaN,V475,392,114,NaN,38,05,S099,V475,T099,T149,S021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,05,V475,S021,S099,T099,T149,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03,NaN,NaN,2,3,100,8,03
2,1 6101 M1093 442411 6D3 2018U7BN C61 113 033 11 0111C61 01 C61 01 11 100 601,1,6,1,01,M,1,093,NaN,44,24,11,NaN,6,D,3,2018,U,7,B,N,NaN,NaN,C61,113,033,NaN,11,01,C61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01,C61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01,NaN,NaN,1,1,100,6,01
3,1 2101 M1090 442411 1W7 2018U7CN A419023 010 37 0311R688 21A419 61I509 03 A419 I509 R688 01 11 100 601,1,2,1,01,M,1,090,NaN,44,24,11,NaN,1,W,7,2018,U,7,C,N,NaN,NaN,A419,023,010,NaN,37,03,R688,A419,I509,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03,A419,I509,R688,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01,NaN,NaN,1,1,100,6,01
4,2 4101 F1031 321205 1S2 2018U7CN K701298 094 30 0911G934 21K701 22R579 31K769 32R688 41D689 61K703 62D649 63E880 08 K701 D649 D689 E880 G934 K703 R579 R688 03 23 100 803,2,4,1,01,F,1,031,NaN,32,12,05,NaN,1,S,2,2018,U,7,C,N,NaN,NaN,K701,298,094,NaN,30,09,G934,K701,R579,K769,R688,D689,K703,D649,E880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,08,K701,D649,D689,E880,G934,K703,R579,R688,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03,NaN,NaN,2,3,100,8,03


In [ ]:
year_2018.dtypes

line_str                object
resident_status         object
education               object
education_flag          object
death_month             object
                         ...  
race_recode_3           object
race_recode_5           object
hispanic                object
hispanic_race_recode    object
race_recode_40          object
Length: 78, dtype: object

### Save file to CSV so we don't have to repeat this!!

In [ ]:
year_2018.to_csv("parsed_data/year_2018.csv")

### Reforming pandas df from csv

In [ ]:
year_2018 = pd.read_csv("parsed_data/year_2018.csv", index_col = "line_num")

/Users/brdurkin/Anaconda/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (42,43,44,62,63,64,65) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/brdurkin/Anaconda/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
year_2018.sample(10)

,line_str,resident_status,education,education_flag,death_month,sex,age_flag,age,age_sub_flag,age_recode_52,age_recode_27,age_recode_12,infant_age_recode_22,place_of_death,marital_status,death_day_of_week,death_year,work_injury,death_manner,disposition_method,autopsy,activity_code,place_of_injury,ucd,ucd_recode_358,ucd_recode_113,ucd_infant_recode_130,ucd_recode_39,mcd_count_E,mcd_1_E,mcd_2_E,mcd_3_E,mcd_4_E,mcd_5_E,mcd_6_E,mcd_7_E,mcd_8_E,mcd_9_E,mcd_10_E,mcd_11_E,mcd_12_E,mcd_13_E,mcd_14_E,mcd_15_E,mcd_16_E,mcd_17_E,mcd_18_E,mcd_19_E,mcd_20_E,mcd_count_R,mcd_1_R,mcd_2_R,mcd_3_R,mcd_4_R,mcd_5_R,mcd_6_R,mcd_7_R,mcd_8_R,mcd_9_R,mcd_10_R,mcd_11_R,mcd_12_R,mcd_13_R,mcd_14_R,mcd_15_R,mcd_16_R,mcd_17_R,mcd_18_R,mcd_19_R,mcd_20_R,race,race_bridged_flag,race_imputed_flag,race_recode_3,race_recode_5,hispanic,hispanic_race_recode,race_recode_40
line_num,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1595821,3 3110 M1031 321205 1D7 2018U7CN B348054 018 37 0411I469 21J189 22B348 61F101 04 B348 F101 I469 J189 01 11 100 601,3,3,1,10,M,1,31,NaN,32,12,5,NaN,1,D,7,2018,U,7.0,C,N,NaN,NaN,B348,54,18,NaN,37,4,I469,J189,B348,F101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,B348,F101,I469,J189,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,1,1,100,6,1
1309725,1 7104 M1070 402009 4M2 2018U7CN I250214 062 21 0211I250 61E149 02 I250 E149 01 11 100 601,1,7,1,4,M,1,70,NaN,40,20,9,NaN,4,M,2,2018,U,7.0,C,N,NaN,NaN,I250,214,62,NaN,21,2,I250,E149,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,I250,E149,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,1,1,100,6,1
1709944,1 5109 F1088 432311 1W3 2018U7BN I48 228 068 22 0411R092 21I639 31I48 61C679 04 I48 C679 I639 R092 01 11 100 601,1,5,1,9,F,1,88,NaN,43,23,11,NaN,1,W,3,2018,U,7.0,B,N,NaN,NaN,I48,228,68,NaN,22,4,R092,I639,I48,C679,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,I48,C679,I639,R092,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,1,1,100,6,1
1345630,1 2109 M1060 381808 4S7 2018U1CY 99X42 420 122 39 0511T519 12X45 13T405 14X42 15T404 05 X42 T404 T405 T519 X45 02 32 100 702,1,2,1,9,M,1,60,NaN,38,18,8,NaN,4,S,7,2018,U,1.0,C,Y,9.0,9.0,X42,420,122,NaN,39,5,T519,X45,T405,X42,T404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,X42,T404,T405,T519,X45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,3,2,100,7,2
2668558,1 4104 F1071 402009 1M6 2018U7EN I634237 070 24 0511J969 21I64 31I634 41I255 61I219 04 I634 I219 I255 J969 01 11 100 601,1,4,1,4,F,1,71,NaN,40,20,9,NaN,1,M,6,2018,U,7.0,E,N,NaN,NaN,I634,237,70,NaN,24,5,J969,I64,I634,I255,I219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,I634,I219,I255,J969,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,1,1,100,6,1
2459300,1 2106 F1077 412110 4W5 2018U7BN I709241 071 25 0311I709 61N185 62E149 02 I709 E142 02 32 100 702,1,2,1,6,F,1,77,NaN,41,21,10,NaN,4,W,5,2018,U,7.0,B,N,NaN,NaN,I709,241,71,NaN,25,3,I709,N185,E149,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,I709,E142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,3,2,100,7,2
1054252,1 4112 F1077 412110 6W2 2018U7RN G309189 052 17 0111G309 01 G309 01 11 100 601,1,4,1,12,F,1,77,NaN,41,21,10,NaN,6,W,2,2018,U,7.0,R,N,NaN,NaN,G309,189,52,NaN,17,1,G309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,G309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,1,1,100,6,1
997081,1 3107 F1075 412110 6D4 2018U7CN K703298 094 30 1011J189 21A419 31J984 41K746 51F109 52E109 61K529 62D649 63I10 64F179 09 K703 A419 D649 E109 F179 I10 J189 J984 K529 01 11 100 601,1,3,1,7,F,1,75,NaN,41,21,10,NaN,6,D,4,2018,U,7.0,C,N,NaN,NaN,K703,298,94,NaN,30,10,J189,A419,J984,K746,F109,E109,K529,D649,I10,F179,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,K703,A419,